In [1]:
#finds site map of entire domain
from usp.tree import sitemap_tree_for_homepage

#for saving and loading objects
import pickle

#for accessing url html
import requests
import urllib.request, urllib.parse, urllib.error
from urllib.request import Request, urlopen

#for html parsing
from bs4 import BeautifulSoup

#anxillary functions/methods
import numpy as np
import pandas as pd
import time
import random
import datetime
from dateutil.tz import tzutc, tzoffset
from tqdm.notebook import tqdm



# Sitemap

In [ ]:
tree = sitemap_tree_for_homepage('https://hongkongfp.com/');

In [ ]:
urls = []
dates = []
for page in tree.all_pages():
    urls.append(page.url)
    dates.append(page.last_modified)
    
#transpose
zippedList =list(zip(urls,dates))

#format into dataframe
df = pd.DataFrame(zippedList, columns = ['urls' ,'date'])

#drop nan for date published
df = df.dropna()
        
pickle.dump(df, open("../Pickles/HKFP_All.p", "wb" ) )


# Data Scrape

In [6]:
#LOAD DATASET OF ALL SCRAPED URLS FROM SCMP
with open("../Pickles/HKFP_All.p", 'rb') as f:
    df = pickle.load(f)
          
#reset df index
df.reset_index(drop = True)

#find articles with "Articles" to exclude photos/video pages
articles_only_urls = []
for i in df.urls:
    if '2018' in i or '2019' in i or '2020' in i:
        articles_only_urls.append(i)
        
author = []
title = []
date = []
summary = []
link = []
broken = []

articles_only_urls = list(dict.fromkeys(articles_only_urls))

In [9]:
#using BS4 to request each video html data
for i in tqdm(articles_only_urls[7674:]):
    try:
        
        time.sleep(2) 
        req = Request(str(i), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')

        by = soup.find("a", {"class": "url fn n"})
        t = soup.find("h1", {"class": "entry-title"})
        d = soup.find("time", {"class": "entry-date published"})
        summ = soup.find("div",{"class":"entry-content"}).select_one("p")


        if by and t and d:
            author.append(by.text)
            title.append(t.text.strip())
            date.append(d.text)
            summary.append(summ.text.strip().replace(u'\xa0', u' '))
            link.append(i)

            if len(link)% 1000 == 0:
                print("Dataset contains " + str(len(link)) + " entries.")
    except:
        print("URL number " + str(articles_only_urls.index(i)) + " is broken: " + i )
        broken.append(i)
        pass

In [12]:
df = pd.DataFrame(list(zip(title, author, date, summary, link)), 
               columns =['Title', 'Author', 'Date', 'Summary','Link']) 
    

with open('../Pickles/HkfpDF.pkl', 'wb') as f:
    pickle.dump(df, f)

# Data Analysis

In [11]:
df

,Title,Author,Date,Summary,Link
0,In Pictures: Hundreds protest against Tuen Mun...,Rachel Wong,"15:09, 24 February 2020",Hundreds joined a “reclaim Tuen Mun Park” prot...,https://hongkongfp.com/2020/02/24/pictures-hun...
1,Traces of coronavirus detected in Hong Kong pa...,Kelly Ho,"13:00, 28 February 2020",A Hong Kong coronavirus patient’s dog has test...,https://hongkongfp.com/2020/02/28/traces-coron...
2,China censors report about how authorities hid...,Global Voices,"09:00, 7 March 2020",By Oiwan Lam,https://hongkongfp.com/2020/03/07/china-censor...
3,Community film screenings across Hong Kong mar...,Rachel Wong,"20:22, 13 March 2020",Film screenings have been held around Hong Kon...,https://hongkongfp.com/2020/03/13/community-fi...
4,Hundreds commemorate 15-year-old Hong Kong stu...,Rachel Wong,"22:00, 23 March 2020",Hundreds attended an event in Tseung Kwan O an...,https://hongkongfp.com/2020/03/23/hundreds-com...
...,...,...,...,...,...
7658,In Pictures: The art of herping in Hong Kong,Chermaine Lee,"14:00, 7 November 2020",By Chermaine Lee and Lawrence Hylton,https://hongkongfp.com/2020/11/07/in-pictures-...
7659,Hong Kong’s arrest of 8 democrats is straight ...,Stephen Vines,"14:21, 7 November 2020",The latest police action against Hong Kong’s p...,https://hongkongfp.com/2020/11/07/why-hong-kon...
7660,Chief Exec. Lam disputes claim journalist’s ar...,Rhoda Kwan,"13:55, 7 November 2020",Hong Kong’s Chief Executive Carrie Lam said Fr...,https://hongkongfp.com/2020/11/07/chief-exec-l...
7661,"Democrat Joe Biden wins US election, ending Do...",AFP,"00:30, 8 November 2020",by Sebastian Smith,https://hongkongfp.com/2020/11/08/breaking-dem...


In [8]:
articles_only_urls[7673:]

['https://hongkongfp.com/newsletter/hkfp-dim-sum-6-11-union-chief-says-govhk-trying-to-silence-teachers-and-the-2020-hong-kong-diversity-list/',
 'https://hongkongfp.com/tag/2018-by-election/',
 'https://hongkongfp.com/tag/2018-policy-address/',
 'https://hongkongfp.com/tag/2019-district-council-election/',
 'https://hongkongfp.com/tag/2019-policy-address/',
 'https://hongkongfp.com/tag/2020-coronavirus/',
 'https://hongkongfp.com/tag/2020-legislative-council-election/',
 'https://hongkongfp.com/tag/2020-policy-address/',
 'https://hongkongfp.com/tag/2020-sony-world-photography-awards/',
 'https://hongkongfp.com/tag/2020-taiwan-elections/',
 'https://hongkongfp.com/tag/2020-us-presidential-elections/',
 'https://hongkongfp.com/tag/2020-legco-elections/',
 'https://hongkongfp.com/tag/budget-2018/',
 'https://hongkongfp.com/tag/budget-2019/',
 'https://hongkongfp.com/tag/budget-2020/',
 'https://hongkongfp.com/tag/diversity-list-2020/',
 'https://hongkongfp.com/tag/gay-games-paris-2018/'